# DocVQA (visual question answering on documents)
## The DONUT model is proposed in the OCR-free Document Understanding Transformer(DONUT) 
## DONUT proposes to solve the following problems.

### CORD (form understanding)
### DocVQA (visual question answering on documents)
### RVL-DIP (document image classification)

In [1]:
# Imports PIL module 
import urllib.request
from PIL import Image

In [10]:
file_path = r""

image = Image.open(file_path).convert("RGB")
image.size

(1152, 648)

In [11]:
from transformers import DonutProcessor, VisionEncoderDecoderModel

processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base-finetuned-docvqa")
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base-finetuned-docvqa")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [12]:
pixel_values = processor(image, return_tensors="pt").pixel_values
print(pixel_values.shape)

torch.Size([1, 3, 2560, 1920])


In [15]:
import torch
import re
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
questions = [
             "what was the Program Name (FROM CURRENT STATE tabel)? "
            ]
# "what was the expected win price ?",
# "who was the delivery leader ?",
task_prompt = "<s_docvqa><s_question>{user_input}</s_question><s_answer>"
for each in questions:
    question = each
    prompt = task_prompt.replace("{user_input}", question)
    decoder_input_ids = processor.tokenizer(prompt, add_special_tokens=False, return_tensors="pt")["input_ids"]
    outputs = model.generate(pixel_values.to(device),
                                decoder_input_ids=decoder_input_ids.to(device),
                                max_length=model.decoder.config.max_position_embeddings,
                                early_stopping=True,
                                pad_token_id=processor.tokenizer.pad_token_id,
                                eos_token_id=processor.tokenizer.eos_token_id,
                                use_cache=True,
                                num_beams=1,
                                bad_words_ids=[[processor.tokenizer.unk_token_id]],
                                return_dict_in_generate=True,
                                output_scores=True)
    seq = processor.batch_decode(outputs.sequences)[0]
    print(processor.token2json(seq))

{'question': 'what was the Program Name (FROM CURRENT STATE tabel)?', 'answer': 'sap migration to asure'}
